In [1]:
import os

os.listdir()


['.DS_Store',
 'merged_all_months.csv',
 'Yusuf_NeuralNet_XGBoost.ipynb',
 '.ipynb_checkpoints']

In [2]:
import pandas as pd

file_path = "merged_all_months.csv"

df = pd.read_csv(file_path)

df.head()


,date,hour,route_code,stop_code,passenger_count,vehicle_type,district,is_outlier,is_peak_hour,day_of_week,is_weekend,mean_passenger_by_route_hour,month
0,2024-08-01,0,MESCIDI SELAM-ARNAVUTKOY-ISTANBUL HAVA LIMANI,OTOYOL,1,1,ATASEHIR,0,0,3,0,1.782609,bilinmeyen_ay
1,2024-08-01,0,MECIDIYEKOY-ISTANBUL HAVALIMANI,OTOYOL,1,1,ATASEHIR,0,0,3,0,1.200000,bilinmeyen_ay
2,2024-08-01,0,KOC UNV. RUMELIFENER KAMPUSU - TAKSIM,OTOYOL,3,1,ATASEHIR,0,0,3,0,1.666667,bilinmeyen_ay
3,2024-08-01,0,HACI OSMAN METRO - RUMELI FENERI,OTOYOL,1,1,ATASEHIR,0,0,3,0,1.538462,bilinmeyen_ay
4,2024-08-01,0,USKUDAR-GUZELTEPE-UMRANIYE DEVLET HASTANESI,OTOYOL,3,1,BAKIRKOY,0,0,3,0,4.258065,bilinmeyen_ay


In [3]:
# 1) Kullanacağımız feature seti
features = [
    'hour',
    'route_code',
    'district',
    'day_of_week',
    'is_weekend',
    'is_peak_hour',
    'mean_passenger_by_route_hour'
]

target = 'vehicle_type'   # Yusuf için sınıflandırma hedefi

# Sadece gerekli kolonlar
df = df[features + [target]]

# Eksik veri varsa temizle
df = df.dropna()

df.head()


,hour,route_code,district,day_of_week,is_weekend,is_peak_hour,mean_passenger_by_route_hour,vehicle_type
0,0,MESCIDI SELAM-ARNAVUTKOY-ISTANBUL HAVA LIMANI,ATASEHIR,3,0,0,1.782609,1
1,0,MECIDIYEKOY-ISTANBUL HAVALIMANI,ATASEHIR,3,0,0,1.200000,1
2,0,KOC UNV. RUMELIFENER KAMPUSU - TAKSIM,ATASEHIR,3,0,0,1.666667,1
3,0,HACI OSMAN METRO - RUMELI FENERI,ATASEHIR,3,0,0,1.538462,1
4,0,USKUDAR-GUZELTEPE-UMRANIYE DEVLET HASTANESI,BAKIRKOY,3,0,0,4.258065,1


In [4]:
from sklearn.model_selection import train_test_split

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape


((2374022, 7), (593506, 7))

In [5]:
from sklearn.preprocessing import LabelEncoder

le_route = LabelEncoder()
le_district = LabelEncoder()

X_train['route_code'] = le_route.fit_transform(X_train['route_code'])
X_test['route_code'] = le_route.transform(X_test['route_code'])

X_train['district'] = le_district.fit_transform(X_train['district'])
X_test['district'] = le_district.transform(X_test['district'])

X_train.head()


ValueError: y contains previously unseen labels: 'KADIKOY-ESENKENT'

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Train-Test'i birleştir
combined = pd.concat([X_train, X_test], axis=0)

# LabelEncoder nesneleri
le_route = LabelEncoder()
le_district = LabelEncoder()

# Encode
combined['route_code'] = le_route.fit_transform(combined['route_code'])
combined['district'] = le_district.fit_transform(combined['district'])

# Tekrar ayır
X_train = combined.iloc[:len(X_train)]
X_test = combined.iloc[len(X_train):]

X_train.head()


TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['int', 'str']

In [7]:
df['route_code'].head(), df['route_code'].apply(type).unique()


(0    MESCIDI SELAM-ARNAVUTKOY-ISTANBUL HAVA LIMANI
 1                  MECIDIYEKOY-ISTANBUL HAVALIMANI
 2            KOC UNV. RUMELIFENER KAMPUSU - TAKSIM
 3                HACI OSMAN METRO -  RUMELI FENERI
 4      USKUDAR-GUZELTEPE-UMRANIYE DEVLET HASTANESI
 Name: route_code, dtype: object,
 array([<class 'str'>], dtype=object))

In [8]:
df['district'].head(), df['district'].apply(type).unique()


(0    ATASEHIR
 1    ATASEHIR
 2    ATASEHIR
 3    ATASEHIR
 4    BAKIRKOY
 Name: district, dtype: object,
 array([<class 'str'>], dtype=object))

In [9]:
from sklearn.preprocessing import LabelEncoder

le_route = LabelEncoder()
le_district = LabelEncoder()

df['route_code'] = le_route.fit_transform(df['route_code'])
df['district'] = le_district.fit_transform(df['district'])

df.head()


,hour,route_code,district,day_of_week,is_weekend,is_peak_hour,mean_passenger_by_route_hour,vehicle_type
0,0,586,2,3,0,0,1.782609,1
1,0,580,2,3,0,0,1.200000,1
2,0,535,2,3,0,0,1.666667,1
3,0,344,2,3,0,0,1.538462,1
4,0,803,6,3,0,0,4.258065,1


In [10]:
X = df.drop('vehicle_type', axis=1)
y = df['vehicle_type']

X.head(), y.head()


(   hour  route_code  district  day_of_week  is_weekend  is_peak_hour  \
 0     0         586         2            3           0             0   
 1     0         580         2            3           0             0   
 2     0         535         2            3           0             0   
 3     0         344         2            3           0             0   
 4     0         803         6            3           0             0   
 
    mean_passenger_by_route_hour  
 0                      1.782609  
 1                      1.200000  
 2                      1.666667  
 3                      1.538462  
 4                      4.258065  ,
 0    1
 1    1
 2    1
 3    1
 4    1
 Name: vehicle_type, dtype: int64)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape


((2374022, 7), (593506, 7))

In [12]:
# === Scaling ===
scaler = StandardScaler()

# Sadece X_train'e fit et → X_test'e sadece transform uygulanır
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled[:5], X_test_scaled[:5]


(array([[ 1.55237529, -1.07439611, -0.85198211, -0.97234228, -0.57578571,
         -1.0124361 , -0.60145586],
        [-0.26022339, -0.42111692,  1.42604048, -1.48769102, -0.57578571,
         -1.0124361 , -0.71185476],
        [-0.62274312, -1.66794692, -0.85198211,  0.57370396, -0.57578571,
          0.98771666, -0.03472516],
        [ 0.64607595,  0.0492441 , -0.85198211,  0.05835522, -0.57578571,
          0.98771666,  0.12246125],
        [ 1.55237529, -0.82428351, -0.85198211,  1.60440145,  1.73675723,
         -1.0124361 , -0.20681154]]),
 array([[-1.34778259,  1.62084719,  1.42604048, -0.45699353, -0.57578571,
         -1.0124361 , -0.81083308],
        [ 0.46481608, -0.122475  , -0.85198211,  1.08905271,  1.73675723,
          0.98771666,  0.96062505],
        [-0.98526286, -0.89521096,  1.42604048, -0.97234228, -0.57578571,
          0.98771666, -0.14316868],
        [-0.44148325,  0.29189066, -0.85198211,  1.08905271,  1.73675723,
         -1.0124361 ,  1.80730311],
        

In [13]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(64, 32),
                    activation='relu',
                    solver='adam',
                    max_iter=20,
                    random_state=42)

mlp.fit(X_train_scaled, y_train)

y_pred_mlp = mlp.predict(X_test_scaled)


/opt/anaconda3/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print("MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("MLP Precision:", precision_score(y_test, y_pred_mlp, average='macro'))
print("MLP Recall:", recall_score(y_test, y_pred_mlp, average='macro'))
print("MLP F1:", f1_score(y_test, y_pred_mlp, average='macro'))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_mlp))


MLP Accuracy: 0.976603437875944
MLP Precision: 0.9248318425066606
MLP Recall: 0.8191472901524435
MLP F1: 0.8652183514615129

Classification Report:

              precision    recall  f1-score   support

           1       0.98      1.00      0.99    543946
           2       0.88      0.65      0.75     19119
           3       0.91      0.81      0.85     30441

    accuracy                           0.98    593506
   macro avg       0.92      0.82      0.87    593506
weighted avg       0.98      0.98      0.98    593506



In [15]:
from xgboost import XGBClassifier


In [16]:
xgb = XGBClassifier(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist"
)

xgb.fit(X_train_scaled, y_train)
y_pred_xgb = xgb.predict(X_test_scaled)


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2], got [1 2 3]

In [17]:
# vehicle_type 1-2-3 → 0-1-2 olarak yeniden etiketleniyor
df['vehicle_type'] = df['vehicle_type'] - 1


In [18]:
X = df.drop(columns=['vehicle_type'])
y = df['vehicle_type']


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [21]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist"
)

xgb.fit(X_train_scaled, y_train)
y_pred_xgb = xgb.predict(X_test_scaled)


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("XGBoost Precision:", precision_score(y_test, y_pred_xgb, average='macro'))
print("XGBoost Recall:", recall_score(y_test, y_pred_xgb, average='macro'))
print("XGBoost F1:", f1_score(y_test, y_pred_xgb, average='macro'))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb))


XGBoost Accuracy: 0.9932856618130229
XGBoost Precision: 0.972440243902965
XGBoost Recall: 0.9479630668524629
XGBoost F1: 0.9597886052077568

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    543946
           1       0.96      0.89      0.92     19119
           2       0.96      0.95      0.96     30441

    accuracy                           0.99    593506
   macro avg       0.97      0.95      0.96    593506
weighted avg       0.99      0.99      0.99    593506



In [23]:
import pandas as pd

results = pd.DataFrame([
    ["MLP (Neural Network)", 0.9766, 0.9248, 0.8191, 0.8652],
    ["XGBoost", 0.9933, 0.9724, 0.9479, 0.9597],
], columns=["Model", "Accuracy", "Precision", "Recall", "F1-score"])

results


,Model,Accuracy,Precision,Recall,F1-score
0,MLP (Neural Network),0.9766,0.9248,0.8191,0.8652
1,XGBoost,0.9933,0.9724,0.9479,0.9597
